In [276]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import time
import matplotlib.pyplot as plt
import sklearn.preprocessing as preprocessing
import sklearn.metrics
import math
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor

In [326]:
x_train = pd.read_csv('x_train_full.csv')
x_val = pd.read_csv('x_val_full.csv')
x_test = pd.read_csv('x_test_full.csv')
y_train = pd.read_csv('./Data with correct player_atts/y_train.csv')
y_val = pd.read_csv('./Data with correct player_atts/y_val.csv')
y_test = pd.read_csv('./Data with correct player_atts/y_test.csv')

In [327]:
label_dict = {1:0, 0:1, -1:2 }
y_train['target'] = y_train['result'].map(label_dict)
y_val['target'] =  y_val['result'].map(label_dict)
y_test['target'] =  y_test['result'].map(label_dict)

In [328]:
(x_train.match_api_id == y_train.match_api_id).sum()

16673

In [288]:
## Rebalance proportion of home wins/draws/home losses
incides = y_train[y_train['result'] == 1].sample(frac = 0.35).index.values
mask_y= ~ y_train.index.isin(indices)
mask_x= ~ x_train.index.isin(indices)
y_train = y_train[mask_y]
x_train = x_train[mask_x]
assert len(x_train) == len(y_train)
assert (x_train.index == y_train.index).sum()
assert (x_train.match_api_id == y_train.match_api_id).sum()

In [329]:
min_max_scaler = preprocessing.MinMaxScaler()

x_train = pd.DataFrame(min_max_scaler.fit_transform(x_train.iloc[:,1:]), index = x_train['match_api_id'], columns = x_train.iloc[:,1:].columns)
x_val = pd.DataFrame(min_max_scaler.fit_transform(x_val.iloc[:,1:]), index = x_val['match_api_id'], columns = x_val.iloc[:,1:].columns)
x_test = pd.DataFrame(min_max_scaler.fit_transform(x_test.iloc[:,1:]), index = x_test['match_api_id'], columns = x_test.iloc[:,1:].columns)

In [330]:
y_train['result'].value_counts(normalize=True)

 1    0.458586
-1    0.289090
 0    0.252324
Name: result, dtype: float64

In [349]:
# Build Network
input_size = len(x_train.columns)
hidden_sizes = [512,256,256,256,64,32]
output_size = 3

# Build a feed-forward network
LinearNN = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.Tanh(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                        nn.ReLU(),
                        nn.Linear(hidden_sizes[2], hidden_sizes[3]),
                        nn.ReLU(),
                         nn.Linear(hidden_sizes[3], hidden_sizes[4]),
                        nn.ReLU(),
                         nn.Linear(hidden_sizes[4], hidden_sizes[5]),
                        nn.ReLU(),
                      nn.Linear(hidden_sizes[5], output_size),
                      nn.Softmax()
                        )


In [392]:
# Hyperparameters

batch_size = 512 #len(x_train)
epochs = 10
learning_rate = 0.001
loss = 0
losses = []
counter = 0

dataset = TensorDataset( Tensor(x_train.values), torch.Tensor(y_train['target'].values) )
train_loader = DataLoader(dataset, batch_size = batch_size, shuffle=False)

optimizer = torch.optim.Adam(LinearNN.parameters(), lr=learning_rate, betas=(0.8,0.999))

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 5 , gamma = 0.1)
mse = torch.nn.MSELoss()
cross = nn.CrossEntropyLoss()

In [ ]:
#Train Model

t = time.time()
for epoch in range(epochs):
    for x, y in iter(train_loader):
        LinearNN.train()
        LinearNN.zero_grad()
        
        y_pred = LinearNN(x)
        loss = cross(y_pred, y.long())
        
        if counter % 500 ==0:
            print('Loss after iteration {}: {}'.format(counter, loss.item()))
        
        losses.append(loss.item())
        
        loss.backward()
        optimizer.step()
        
        counter+=1 
    counter+=1
    scheduler.step()
time.time()-t        
        
        
print('Elapsed time: {} s'.format(time.time()-t))    
print(loss.item())  
plt.plot(range(len(losses)), losses)
plt.show()

Loss after iteration 0: 1.1061350107192993


In [ ]:
##Eval 

LinearNN.eval()
with torch.no_grad():
    val_pred = LinearNN(Tensor(x_val.values))
    
y_val_pred = pd.Series(val_pred.max(1).indices).map({0:1,1:0,2:-1})
sklearn.metrics.accuracy_score(y_val_pred, y_val['result'])

In [ ]:
y_val_pred.value_counts()

In [338]:
y_pred

tensor([[0.6496, 0.1033, 0.2471],
        [0.6608, 0.0983, 0.2408],
        [0.6526, 0.1020, 0.2454],
        [0.6572, 0.0999, 0.2428],
        [0.6951, 0.0838, 0.2212],
        [0.6632, 0.0973, 0.2395],
        [0.6541, 0.1013, 0.2446],
        [0.6778, 0.0910, 0.2312],
        [0.6797, 0.0902, 0.2301],
        [0.6567, 0.1002, 0.2431],
        [0.6647, 0.0966, 0.2386],
        [0.6747, 0.0923, 0.2330],
        [0.6627, 0.0976, 0.2398],
        [0.6915, 0.0853, 0.2233],
        [0.6693, 0.0946, 0.2360],
        [0.6497, 0.1033, 0.2470],
        [0.6649, 0.0966, 0.2385],
        [0.6857, 0.0877, 0.2266],
        [0.6599, 0.0987, 0.2413],
        [0.6943, 0.0841, 0.2216],
        [0.6542, 0.1013, 0.2445],
        [0.6806, 0.0898, 0.2296],
        [0.6762, 0.0917, 0.2321],
        [0.6711, 0.0939, 0.2350],
        [0.6647, 0.0967, 0.2387],
        [0.6733, 0.0929, 0.2338],
        [0.6740, 0.0926, 0.2334],
        [0.6710, 0.0939, 0.2351],
        [0.6925, 0.0848, 0.2227],
        [0.677